HANOI4

HANOI4B의 문제는 없고 비슷한 문제인 HANOI4가 존재합니다.  
[algospot의 HANOI4 링크](https://algospot.com/judge/problem/read/HANOI4)

### 문제  
![](http://algospot.com/media/judge-attachments/8cb6ad23fcf3eb42d978d8da99e9ace3/Hanoi.jpg)  
하노이의 탑은 세 개의 기둥에 꽂혀 있는 N개의 원반을 가지고 하는 게임입니다. N개의 원반은 크기가 모두 다르며, 게임의 시작 때는 그림과 같이 맨 왼쪽의 기둥에 모두 크기 순서대로 꽂혀 있습니다. 게임의 목적은 모든 원반을 맨 오른쪽 기둥으로 옮기는 것입니다.  
  
원반을 움직일 때는 다음과 같은 규칙을 따라야 합니다.  
  
한 번에 한 개의 원반만을 움직일 수 있습니다. 원반을 기둥이 아닌 다른 곳에 내려놓거나, 원반을 하나 들고 있으면서 다른 원반을 움직일 수는 없습니다.  
작은 원반 위에 큰 원반을 올려놓아서는 안 됩니다.  
세 개의 기둥이 있을 때 하노이의 탑 문제에는 간단한 해법이 있음은 잘 알려진 사실입니다. 그러면 기둥이 네 개가 있으면 어떨까요? 네 개의 기둥에 원반들이 자유롭게 배치되어 있을 때, 모든 원반을 오른쪽 기둥으로 옮겨 놓기 위해 필요한 최소한의 움직임 수를 계산하는 프로그램을 작성하세요.

  
  
### 출력  
각 테스트 케이스마다 모든 원반을 맨 오른쪽 기둥으로 옮겨 놓기 위한 최소 움직임의 수를 출력합니다. 항상 모든 원반을 맨 오른쪽 기둥으로 옮겨 놓을 수 있다고 가정해도 좋습니다.    

getting 함수의  
state : 현재 원반 배열 상태
index : 원반의 index (0 ~ n-1)

setting 함수의  
state : 현재 원반 배열 상태
index : 원반의 index (0 ~ n-1)
value : 기둥의 index

solution 함수의  
n : 원반의 총 수  
info : 각 기둥별 원반의 정보 리스트

In [1]:
from queue import Queue

# 각 state는 비트 마스크를 이용해서, 각 원반 별로 기둥의 정보를 주도록 한다.
# 0번 기둥 -> 00, 1번 기둥 -> 01, 2번 기둥 -> 10, 3번 기둥 -> 11
# 그렇게 원반 인덱스 역순으로 포함되는 기둥에 대한 정보를 붙여준다.
# ex) n = 3일 경우,    1번 원반 -> 2번 기둥,  2번 원반 -> 3번 기둥,  3번 원반 -> 1번 기둥
#     011011 의 값이 나오며 이를 10진수로 변환하면 27이 된다. 

MAX_DISCS = 12
c = [0 for _ in range(1<<(MAX_DISCS * 2))]

def getting(state, index):
    # 현재의 state에서 index번 원반을 지니는 기둥의 번호 반환
    return (state >> (index * 2)) & 3

def setting(state, index, value):
    # 현재의 state에서 index번 원반을 value 기둥으로 옮긴 상태 세팅
    return (state & ~(3 << (index * 2))) | (value << (index * 2))

def sgn(x): # bidirectional으로 양방향으로 너비우선 탐색을 진행하기 때문에 방향을 반환
    if not x:
        return 0
    elif x > 0:
        return 1
    return -1

def incr(x): # 방향에 따라 값을 더할건지 뺄건지 확인, 결국 절댓값을 1 증가시킨다.
    if x<0:
        return x-1
    return x+1

def bidirectional_bfs(discs, begin, end):
    if begin == end: return 0 # 만약 모두 맨 마지막 기둥으로 옮겨져 있을 때 0 반환
    q = Queue() # 상태를 집어넣을 큐 생성
    c = [0 for _ in range(1<<(MAX_DISCS * 2))] # 최대 원반 수 12개에 대해서 모두 상태를 집어 넣을 수 있도록(각 원반당 2비트 소요) 크기에 맞게 리스트 생성
    # 해당 리스트에는 상태에 따라 어떠한 방향으로 접근해나갈 것인지에 대한 정보를 가진다.
    q.put(begin) # 현 상태를 큐에 넣는다.
    c[begin] = 1 # 정방향으로 진행
    q.put(end) # 마지막 상태를 큐에 넣는다.
    c[end] = -1 # 역방향으로 진행
    while not q.empty(): # 큐가 빌때까지 계속 반복
        here = q.get() # 현재 상태 큐에서 반환
        top = [-1,-1,-1,-1] # 각 기둥별로 가장 위에 있는 원반의 인덱스 저장할 리스트
        for i in range(discs-1, -1, -1): # 각 원반에 대해 역순으로 접근
            top[getting(here, i)] = i # 현 상태에서 i번 원반을 지니는 기둥 번호 반환
        for i in range(4): # 각 기둥에 대해서
            if top[i] != -1: # 만약 i번 기둥에 원반이 1개 이상 있다면,
                for j in range(4):
                    if i != j and (top[j] == -1 or top[j] > top[i]): # i를 제외한 기둥들 중에 i보다 더 높은 인덱스 원반을 지니는 기둥 j에 대해서
                        there = setting(here, top[i], j) # i 기둥의 상위에 있는 원반을 기둥 j로 옮긴 상태를 반환
                        if c[there] == 0: # 만약 그 상태가 0이라면
                            c[there] = incr(c[here]) # 옮기기 전의 상태의 접근방향을 따른다.
                            q.put(there) # 그리고 바뀐 상태를 큐에 다시 넣는다.
                        elif sgn(c[there]) != sgn(c[here]): # 상태가 0이 아니고, 양 상태의 접근 방향이 다를 경우
                            return abs(c[there]) + abs(c[here]) - 1 
                        # 이는 정답이므로 여태까지 상태를 바꾼 횟수들을 합하여 반환한다.
    return -1
    
def solution(n, info):
    begin = 0 # 시작 상태
    end = 0 # 끝 상태
    num = [0 for _ in range(n)] # 각 원반별 포함되어있는 기둥의 인덱스를 저장할 리스트
    for i in range(len(info)):
        for j in info[i]:
            num[j-1] = i # 각 원반에 대해서 포함되어 있는 기둥을 저장한다.

    for i in range(len(num)-1, -1, -1): # 그리고 저장된 기둥 값에 대해서 
        begin *= 4
        end *= 4
        begin += num[i] # 각 기둥에 대한 값을 더해준다.
        end += 3 # 마지막 상태는 모두 3번 기둥에 자리 잡아야하기 때문에 3을 더해준다.
    return bidirectional_bfs(n, begin, end) # 그리고 최소 이동 횟수 반환

In [2]:
n = 5
info = [[1], [3], [5,4], [2]]

In [3]:
n = 3
info = [[2], [], [3,1], []]

In [4]:
solution(n, info)

4

In [5]:
1 << 24

16777216